In [1]:
from numba import jit, njit

In [2]:
# Distance imports
from functools import partial

import numpy as np
from scipy._lib._util import _asarray_validated
from scipy.spatial import _distance_wrap
from scipy.spatial.distance import _args_to_kwargs_xdist, _METRIC_ALIAS, _filter_deprecated_kwargs, _METRICS, \
    mahalanobis, wminkowski, minkowski, seuclidean, _validate_cdist_input, _select_weighted_metric, _C_WEIGHTED_METRICS, \
    _TEST_METRICS, squareform, _validate_pdist_input, jensenshannon, _convert_to_type

_convert_to_double = partial(_convert_to_type, out_type=np.double)

In [3]:
import time

import numpy as np

# BO import
from bayes_opt import BayesianOptimization
from bayes_opt.util import acq_max
from bayes_opt import UtilityFunction
from sklearn.gaussian_process.kernels import RBF

# Kendall distance import
from scipy.stats._stats import _kendall_dis

# Kernel imports
from sklearn.gaussian_process.kernels import StationaryKernelMixin, NormalizedKernelMixin, Kernel, Hyperparameter, _check_length_scale, _approx_fprime

# Distance imports
from scipy.spatial.distance import squareform

# Other imports
import warnings

In [4]:
# Imports with the problem
from scipy.spatial.distance import pdist, cdist

# Some definitions necessary for the program

In [5]:
def kendall_distance(x,y):
    perm = np.argsort(y)  # sort on y and convert y to dense ranks
    x, y = x[perm], y[perm]
    y = np.r_[True, y[1:] != y[:-1]].cumsum(dtype=np.intp)

    # stable sort on x and convert x to dense ranks
    perm = np.argsort(x, kind='mergesort')
    x, y = x[perm], y[perm]
    x = np.r_[True, x[1:] != x[:-1]].cumsum(dtype=np.intp)

    dis = _kendall_dis(x, y)  # discordant pairs
    return dis

In [6]:
def random_key(v):
    permutation = np.argsort(v)
    return permutation

In [7]:
def black_box_function(**kwargs):
    data = np.fromiter(kwargs.values(), dtype=float)
    return np.sum(data)

In [8]:
def generate_bounds(n, lower_bound=0, upper_bound=1):
    i = 0
    pbounds = {}
    while i < n:
        xi = 'x' + str(i)
        pbounds[xi] = (lower_bound, upper_bound)
        i += 1
    return pbounds

# Definition of kernel.
### Here the problem appears

In [9]:
class PermutationRBF(StationaryKernelMixin, NormalizedKernelMixin, Kernel):
    def __init__(self, alpha=1.0, alpha_bounds=(1e-5, 1e5)):
        self.alpha = alpha
        self.alpha_bounds = alpha_bounds

    @property
    def anisotropic(self):
        return np.iterable(self.alpha) and len(self.alpha) > 1

    @property
    def hyperparameter_length_scale(self):
        if self.anisotropic:
            return Hyperparameter("length_scale", "numeric",
                                  self.alpha_bounds,
                                  len(self.alpha))
        return Hyperparameter(
            "alpha", "numeric", self.alpha_bounds)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        alpha = _check_length_scale(X, self.alpha)
        if Y is None:
            dists = custom_pdist(X / alpha, kendall_distance)
            K = np.exp(-.5 * dists)
            # convert from upper-triangular matrix to square matrix
            K = squareform(K)
            np.fill_diagonal(K, 1)
        else:
            if eval_gradient:
                raise ValueError(
                    "Gradient can only be evaluated when Y is None.")
            dists = custom_cdist(X / alpha, Y / alpha, kendall_distance)
            K = np.exp(-.5 * dists)
        if eval_gradient:
            if self.hyperparameter_length_scale.fixed:
                # Hyperparameter l kept fixed
                return K, np.empty((X.shape[0], X.shape[0], 0))
            elif not self.anisotropic or alpha.shape[0] == 1:
                K_gradient = \
                    (K * squareform(dists))[:, :, np.newaxis]
                return K, K_gradient
            elif self.anisotropic:
                # We need to recompute the pairwise dimension-wise distances
                K_gradient = (X[:, np.newaxis, :] - X[np.newaxis, :, :]) ** 2 \
                    / (alpha ** 2)
                K_gradient *= K[..., np.newaxis]
                return K, K_gradient
        else:
            return K

In [10]:
class RBF(StationaryKernelMixin, NormalizedKernelMixin, Kernel):
    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5)):
        self.length_scale = length_scale
        self.length_scale_bounds = length_scale_bounds

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    @property
    def hyperparameter_length_scale(self):
        if self.anisotropic:
            return Hyperparameter("length_scale", "numeric",
                                  self.length_scale_bounds,
                                  len(self.length_scale))
        return Hyperparameter(
            "length_scale", "numeric", self.length_scale_bounds)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        length_scale = _check_length_scale(X, self.length_scale)
        if Y is None:
            dists = pdist(X / length_scale, metric='sqeuclidean')
            K = np.exp(-.5 * dists)
            # convert from upper-triangular matrix to square matrix
            K = squareform(K)
            np.fill_diagonal(K, 1)
        else:
            if eval_gradient:
                raise ValueError(
                    "Gradient can only be evaluated when Y is None.")
            dists = cdist(X / length_scale, Y / length_scale,
                          metric='sqeuclidean')
            K = np.exp(-.5 * dists)

        if eval_gradient:
            if self.hyperparameter_length_scale.fixed:
                # Hyperparameter l kept fixed
                return K, np.empty((X.shape[0], X.shape[0], 0))
            elif not self.anisotropic or length_scale.shape[0] == 1:
                K_gradient = \
                    (K * squareform(dists))[:, :, np.newaxis]
                return K, K_gradient
            elif self.anisotropic:
                # We need to recompute the pairwise dimension-wise distances
                K_gradient = (X[:, np.newaxis, :] - X[np.newaxis, :, :]) ** 2 \
                    / (length_scale ** 2)
                K_gradient *= K[..., np.newaxis]
                return K, K_gradient
        else:
            return K

    def __repr__(self):
        if self.anisotropic:
            return "{0}(length_scale=[{1}])".format(
                self.__class__.__name__, ", ".join(map("{0:.3g}".format,
                                                   self.length_scale)))
        else:  # isotropic
            return "{0}(length_scale={1:.3g})".format(
                self.__class__.__name__, np.ravel(self.length_scale)[0])

In [11]:
def custom_pdist(X, metric='euclidean', *args, **kwargs):
    X = _asarray_validated(X, sparse_ok=False, objects_ok=True, mask_ok=True,
                           check_finite=False)
    kwargs = _args_to_kwargs_xdist(args, kwargs, metric, "pdist")

    X = np.asarray(X, order='c')

    s = X.shape
    if len(s) != 2:
        raise ValueError('A 2-dimensional array must be passed.')

    m, n = s
    out = kwargs.pop("out", None)
    if out is None:
        dm = np.empty((m * (m - 1)) // 2, dtype=np.double)
    else:
        if out.shape != (m * (m - 1) // 2,):
            raise ValueError("output array has incorrect shape.")
        if not out.flags.c_contiguous:
            raise ValueError("Output array must be C-contiguous.")
        if out.dtype != np.double:
            raise ValueError("Output array must be double type.")
        dm = out

    # compute blocklist for deprecated kwargs
    if(metric in _METRICS['jensenshannon'].aka
       or metric == 'test_jensenshannon' or metric == jensenshannon):
        kwargs_blocklist = ["p", "w", "V", "VI"]

    elif(metric in _METRICS['minkowski'].aka
         or metric in _METRICS['wminkowski'].aka
         or metric in ['test_minkowski', 'test_wminkowski']
         or metric in [minkowski, wminkowski]):
        kwargs_blocklist = ["V", "VI"]

    elif(metric in _METRICS['seuclidean'].aka or
         metric == 'test_seuclidean' or metric == seuclidean):
        kwargs_blocklist = ["p", "w", "VI"]

    elif(metric in _METRICS['mahalanobis'].aka
         or metric == 'test_mahalanobis' or metric == mahalanobis):
        kwargs_blocklist = ["p", "w", "V"]

    else:
        kwargs_blocklist = ["p", "V", "VI"]

    _filter_deprecated_kwargs(kwargs, kwargs_blocklist)

    if callable(metric):
        mstr = getattr(metric, '__name__', 'UnknownCustomMetric')
        metric_name = _METRIC_ALIAS.get(mstr, None)

        if metric_name is not None:
            X, typ, kwargs = _validate_pdist_input(X, m, n,
                                                   metric_name, **kwargs)

        dm = calculate_pdist_dm(metric,dm,m,X,**kwargs)

    elif isinstance(metric, str):
        mstr = metric.lower()

        mstr, kwargs = _select_weighted_metric(mstr, kwargs, out)

        metric_name = _METRIC_ALIAS.get(mstr, None)

        if metric_name is not None:
            X, typ, kwargs = _validate_pdist_input(X, m, n,
                                                   metric_name, **kwargs)

            if 'w' in kwargs:
                metric_name = _C_WEIGHTED_METRICS.get(metric_name, metric_name)

            # get pdist wrapper
            pdist_fn = getattr(_distance_wrap,
                               "pdist_%s_%s_wrap" % (metric_name, typ))
            pdist_fn(X, dm, **kwargs)
            return dm

        elif mstr in ['old_cosine', 'old_cos']:
            warnings.warn('"old_cosine" is deprecated and will be removed in '
                          'a future version. Use "cosine" instead.',
                          DeprecationWarning)
            X = _convert_to_double(X)
            norms = np.einsum('ij,ij->i', X, X, dtype=np.double)
            np.sqrt(norms, out=norms)
            nV = norms.reshape(m, 1)
            # The numerator u * v
            nm = np.dot(X, X.T)
            # The denom. ||u||*||v||
            de = np.dot(nV, nV.T)
            dm = 1.0 - (nm / de)
            dm[range(0, m), range(0, m)] = 0.0
            dm = squareform(dm)
        elif mstr.startswith("test_"):
            if mstr in _TEST_METRICS:
                dm = pdist(X, _TEST_METRICS[mstr], **kwargs)
            else:
                raise ValueError('Unknown "Test" Distance Metric: %s' % mstr[5:])
        else:
            raise ValueError('Unknown Distance Metric: %s' % mstr)
    else:
        raise TypeError('2nd argument metric must be a string identifier '
                        'or a function.')
    return dm

In [12]:
def custom_cdist(XA, XB, metric='euclidean', *args, **kwargs):
    kwargs = _args_to_kwargs_xdist(args, kwargs, metric, "cdist")

    XA = np.asarray(XA, order='c')
    XB = np.asarray(XB, order='c')

    s = XA.shape
    sB = XB.shape

    if len(s) != 2:
        raise ValueError('XA must be a 2-dimensional array.')
    if len(sB) != 2:
        raise ValueError('XB must be a 2-dimensional array.')
    if s[1] != sB[1]:
        raise ValueError('XA and XB must have the same number of columns '
                         '(i.e. feature dimension.)')

    mA = s[0]
    mB = sB[0]
    n = s[1]
    out = kwargs.pop("out", None)
    if out is None:
        dm = np.empty((mA, mB), dtype=np.double)
    else:
        if out.shape != (mA, mB):
            raise ValueError("Output array has incorrect shape.")
        if not out.flags.c_contiguous:
            raise ValueError("Output array must be C-contiguous.")
        if out.dtype != np.double:
            raise ValueError("Output array must be double type.")
        dm = out

    # compute blocklist for deprecated kwargs
    if(metric in _METRICS['minkowski'].aka or
       metric in _METRICS['wminkowski'].aka or
       metric in ['test_minkowski', 'test_wminkowski'] or
       metric in [minkowski, wminkowski]):
        kwargs_blocklist = ["V", "VI"]
    elif(metric in _METRICS['seuclidean'].aka or
         metric == 'test_seuclidean' or metric == seuclidean):
        kwargs_blocklist = ["p", "w", "VI"]
    elif(metric in _METRICS['mahalanobis'].aka or
         metric == 'test_mahalanobis' or metric == mahalanobis):
        kwargs_blocklist = ["p", "w", "V"]
    else:
        kwargs_blocklist = ["p", "V", "VI"]

    _filter_deprecated_kwargs(kwargs, kwargs_blocklist)

    if callable(metric):

        mstr = getattr(metric, '__name__', 'Unknown')
        metric_name = _METRIC_ALIAS.get(mstr, None)

        XA, XB, typ, kwargs = _validate_cdist_input(XA, XB, mA, mB, n,
                                                    metric_name, **kwargs)

        dm = calculate_cdist_dm(metric,dm,mA,mB,XA,XB,**kwargs)

    elif isinstance(metric, str):
        mstr = metric.lower()

        mstr, kwargs = _select_weighted_metric(mstr, kwargs, out)

        metric_name = _METRIC_ALIAS.get(mstr, None)
        if metric_name is not None:
            XA, XB, typ, kwargs = _validate_cdist_input(XA, XB, mA, mB, n,
                                                        metric_name, **kwargs)

            if 'w' in kwargs:
                metric_name = _C_WEIGHTED_METRICS.get(metric_name, metric_name)

            # get cdist wrapper
            cdist_fn = getattr(_distance_wrap,
                               "cdist_%s_%s_wrap" % (metric_name, typ))
            cdist_fn(XA, XB, dm, **kwargs)
            return dm

        elif mstr.startswith("test_"):
            if mstr in _TEST_METRICS:
                dm = cdist(XA, XB, _TEST_METRICS[mstr], **kwargs)
            else:
                raise ValueError('Unknown "Test" Distance Metric: %s' % mstr[5:])
        else:
            raise ValueError('Unknown Distance Metric: %s' % mstr)
    else:
        raise TypeError('2nd argument metric must be a string identifier '
                        'or a function.')
    return dm

In [13]:
@jit
def calculate_pdist_dm(metric,dm,m,X,**kwargs):
    k = 0
    for i in range(0, m - 1):
        for j in range(i + 1, m):
            dm[k] = metric(X[i], X[j], **kwargs)
            k = k + 1
    return dm

In [14]:
@jit
def calculate_cdist_dm(metric,dm,mA,mB,XA,XB,**kwargs):
    for i in range(0, mA):
        for j in range(0, mB):
            dm[i, j] = metric(XA[i], XB[j], **kwargs)
    return dm

In [15]:
class MyBayesianOptimization(BayesianOptimization):
    def suggest(self, utility_function):
        """Most promissing point to probe next"""
        if len(self._space) == 0:
            return self._space.array_to_params(self._space.random_sample())

        # Sklearn's GP throws a large number of warnings at times, but
        # we don't really need to see them here.
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self._gp.fit(self._space.params, self._space.target)

        # Finding argmax of the acquisition function.
        suggestion = acq_max(
            ac=utility_function.utility,
            gp=self._gp,
            y_max=self._space.target.max(),
            bounds=self._space.bounds,
            random_state=self._random_state,
            n_warmup=10000,
            n_iter=0 # This is the only change of the method
        )

        return self._space.array_to_params(suggestion)

# Program starts here

### Using my custom kernel (slow)

In [16]:
n = 20
seed = 0
it = 5
kappa = 2.5
xi = 0.0

In [17]:
# Bounds of each variable
pbounds = generate_bounds(n)

# Bayesian Optimizer
optimizer = MyBayesianOptimization(
    f=None,
    pbounds=pbounds,
    verbose=2,
    random_state=seed,
)

# Set the Kernel
optimizer.set_gp_params(kernel=PermutationRBF())

# Set the Acquisition function
utility = UtilityFunction(kind="ucb", kappa=kappa, xi=xi)

# Bayesian Optimization with Gaussian Process
for i in range(it):
    t_ini = time.time()
    next_point = optimizer.suggest(utility)
    t_end = time.time() - t_ini
    print('iteration: ',i)
    print('time: ', t_end)
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target=target)

iteration:  0
time:  0.00023889541625976562


TypeError: not enough arguments: expected 5, got 4

### Using RBF kernel (very fast)

In [18]:
n = 20
seed = 0
it = 5
kappa = 2.5
xi = 0.0

In [19]:
# Bounds of each variable
pbounds = generate_bounds(n)

# Bayesian Optimizer
optimizer = MyBayesianOptimization(
    f=None,
    pbounds=pbounds,
    verbose=2,
    random_state=seed,
)

# Set the Kernel
optimizer.set_gp_params(kernel=RBF())

# Set the Acquisition function
utility = UtilityFunction(kind="ucb", kappa=kappa, xi=xi)

# Bayesian Optimization with Gaussian Process
for i in range(it):
    t_ini = time.time()
    next_point = optimizer.suggest(utility)
    t_end = time.time() - t_ini
    print('iteration: ',i)
    print('time: ', t_end)
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target=target)

iteration:  0
time:  0.0004973411560058594
iteration:  1
time:  0.03358173370361328
iteration:  2
time:  0.01837444305419922
iteration:  3
time:  0.017252445220947266
iteration:  4
time:  0.010966300964355469


# Numba test

In [20]:
def test_numba():
    k = 0
    for i in range(10000):
        for j in range(10000):
            k += 1
    return k

In [21]:
t_ini = time.time()
test_numba()
t_end = time.time() - t_ini
t_end

3.9768311977386475

In [22]:
@jit
def test_numba_jit():
    k = 0
    for i in range(10000):
        for j in range(10000):
            k += 1
    return k

In [23]:
t_ini = time.time()
test_numba_jit()
t_end = time.time() - t_ini
t_end

0.12893915176391602